In [30]:
import osmnx as ox
import folium
import geopandas as gpd
# 峠道
graph = ox.graph_from_point(center_point=(35.334446, 136.990590)
                                , network_type='drive'
                                , dist=1000
                                , simplify=True
                                , custom_filter='["highway"~"tertiary|secondary|primary"]')

# グラフデータをGeoDataFrameに変換
gdf_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)
gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

def get_nearest_node_by_coordinates(graph, latitude, longitude):
    nearest_node = ox.distance.nearest_nodes(
        graph, longitude, latitude, return_dist=False
    )
    return nearest_node

# 中央付近のノードを取得
center_node = get_nearest_node_by_coordinates(graph, 35.334446, 136.990590)

# 探索を開始するエッジを取得
start_edge = gdf_edges.loc[(gdf_edges.index.get_level_values(0) == center_node)]

line_dicts = []
line_dict = {'total_distance': 0, 'start_node': center_node, 'connected_nodes': [center_node] }

max_distance = 1000

# 全エッジ数を出力
print('start node: ', center_node)
print('edge count: ', len(gdf_edges))

# 指定したrefのエッジを先頭に来るように並び替える
def sort_edge_by_ref(edge: gpd.GeoDataFrame, ref):
  for index, row in edge.iterrows():
    # rowがlist型の場合
    if(isinstance(row.ref, list)):
      # listの中にrefが存在する場合は先頭に来るように並び替える
      if(row.ref.count(ref) > 0):
        # 先頭の行と、indexを含むキーの行を入れ替える
        edge.loc[index], edge.loc[edge.index[0]] = edge.loc[edge.index[0]], edge.loc[index]
    else:
      if(row.ref == ref):
        edge.loc[index], edge.loc[edge.index[0]] = edge.loc[edge.index[0]], edge.loc[index]
  return edge

# 探索処理
def search(line_dicts, edge, line_dict, previous_node):
  print('node_count: ', len(edge))
  # refが同じ物を先頭に来るように並び替え, 数値ではなく配列で道路番号が複数入っている場合があってそこでエラーになる。
  # edge = edge.sort_values('ref')
  edge = sort_edge_by_ref(edge, edge.iloc[0].ref)
  
  for index, row in edge.iterrows():
    st_node = index[0]
    ed_node = index[1]
    ref = edge.iloc[0].ref
    # 直前のノードと同じ場合はスキップ
    # print('pre_node: ', previous_node)
    if(ed_node == previous_node):
      # print('skip')
      continue
    print('st: ', st_node, 'ed: ', ed_node, 'ref', ref)
    
    # refが同じ場合はラインが1000mを超えるまで探索を続ける。
    if(row.ref == edge.iloc[0].ref):
      # ノードを追加
      line_dict['connected_nodes'].append(ed_node)
      # 距離を加算
      line_dict['total_distance'] += row.length
      # 1000mを超えていたらラインを確定させる
      if(line_dict['total_distance'] > max_distance):
        line_dicts.append(line_dict)
        line_dict = {'total_distance': 0, 'start_node': st_node, 'connected_nodes': [st_node] }
    else:
      line_dict = {'total_distance': 0, 'start_node': st_node, 'connected_nodes': [st_node] }
    # 子ノードが存在する場合は再帰的に探索を続ける
    if(len(gdf_edges.loc[(gdf_edges.index.get_level_values(0) == ed_node)]) > 0):
      r_line_dicts, r_line_dict = search(line_dicts, gdf_edges.loc[(gdf_edges.index.get_level_values(0) == ed_node)], line_dict, st_node)
      line_dicts = r_line_dicts
      line_dict = r_line_dict
  return line_dicts, line_dict
    
r_line_dicts, r_line_dict = search(line_dicts, start_edge, line_dict, 0)

start node:  1072275362
edge count:  9
node_count:  1
st:  1072275362 ed:  426691834 ref 16
node_count:  3
st:  426691834 ed:  3822987058 ref 453
node_count:  3
st:  3822987058 ed:  10892645426 ref 453
node_count:  1
st:  3822987058 ed:  10892640497 ref 453
node_count:  1
st:  10892640497 ed:  426691834 ref 16
node_count:  3
st:  426691834 ed:  3822987058 ref 453
node_count:  3
st:  3822987058 ed:  10892645426 ref 453
node_count:  1
st:  3822987058 ed:  10892640497 ref 453
node_count:  1
st:  10892640497 ed:  426691834 ref 16
node_count:  3
st:  426691834 ed:  3822987058 ref 453
node_count:  3
st:  3822987058 ed:  10892645426 ref 453
node_count:  1
st:  3822987058 ed:  10892640497 ref 453
node_count:  1
st:  10892640497 ed:  426691834 ref 16
node_count:  3
st:  426691834 ed:  3822987058 ref 453
node_count:  3
st:  3822987058 ed:  10892645426 ref 453
node_count:  1
st:  3822987058 ed:  10892640497 ref 453
node_count:  1
st:  10892640497 ed:  426691834 ref 16
node_count:  3
st:  42669183

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1758, in _pydevd_bundle.pydevd_cython.ThreadTracer.__call__
RecursionError: maximum recursion depth exceeded


node_count:  1
st:  3822987058 ed:  10892640497 ref 453
node_count:  1
st:  10892640497 ed:  426691834 ref 16
node_count:  3
st:  426691834 ed:  3822987058 ref 453
node_count:  3
st:  3822987058 ed:  10892645426 ref 453
node_count:  1
st:  3822987058 ed:  10892640497 ref 453
node_count:  1
st:  10892640497 ed:  426691834 ref 16
node_count:  3
st:  426691834 ed:  3822987058 ref 453
node_count:  3
st:  3822987058 ed:  10892645426 ref 453
node_count:  1
st:  3822987058 ed:  10892640497 ref 453
node_count:  1
st:  10892640497 ed:  426691834 ref 16
node_count:  3
st:  426691834 ed:  3822987058 ref 453
node_count:  3
st:  3822987058 ed:  10892645426 ref 453
node_count:  1
st:  3822987058 ed:  10892640497 ref 453
node_count:  1
st:  10892640497 ed:  426691834 ref 16
node_count:  3
st:  426691834 ed:  3822987058 ref 453
node_count:  3
st:  3822987058 ed:  10892645426 ref 453
node_count:  1
st:  3822987058 ed:  10892640497 ref 453
node_count:  1
st:  10892640497 ed:  426691834 ref 16
node_count

: 

: 